In [117]:
%load_ext autoreload
%autoreload 2
import pybedtools
import numpy as np
from intervals import bed_intersection_labels, multibed_intersection_labels, bin_bed

atf1_k562_bed = '/mnt/data/ENCODE/peaks_spp/mar2012/distinct/idrOptimal/\
wgEncodeSydhTfbsK562Atf106325StdAlnRep0.bam_VS_wgEncodeSydhTfbsK562InputStdAlnRep0.bam.regionPeak.gz'

bcl3_k562_bed = '/mnt/data/ENCODE/peaks_spp/mar2012/distinct/idrOptimal/\
wgEncodeHaibTfbsK562Bcl3Pcr1xAlnRep0.bam_VS_wgEncodeHaibTfbsK562RxlchPcr1xAlnRep0.bam.regionPeak.gz'

dnase_562_bed = '/mnt/data/epigenomeRoadmap/peaks/consolidated/narrowPeak/E123-DNase.macs2.narrowPeak.gz'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def add_flanks_to_bed(bed, flank_size):
    pass

In [4]:
atf1_k562 = pybedtools.BedTool(atf1_k562_bed)
bcl3_k562 = pybedtools.BedTool(bcl3_k562_bed)
dnase_k562 = pybedtools.BedTool(dnase_562_bed)

In [44]:
atf1_labels = bed_intersection_labels(dnase_k562, atf1_k562)

In [45]:
bcl3_labels = bed_intersection_labels(dnase_k562, bcl3_k562)

In [46]:
atf1_labels.sum()

13653

In [7]:
atf1_labels.shape

(341918, 1)

In [8]:
bcl3_labels.shape

(341918, 1)

In [9]:
all_labels = multibed_intersection_labels(dnase_k562, [atf1_k562, bcl3_k562])

In [10]:
all_labels.shape

(341918, 2)

In [118]:
#binned_dnase = dnase_k562.window_maker(dnase_k562, w=1000, s=600, i="winnum")
binned_dnase = bin_bed(dnase_k562, bin_size=1000, stride=600)

In [119]:
for interval in binned_dnase[:10]:
    print interval

chrM	0	1000	1

chrM	1208	2208	1

chrM	1808	2808	2

chrM	2408	3408	3

chrM	3008	4008	4

chrM	8866	9866	1

chrM	9466	10466	2

chrM	10066	11066	3

chrM	10666	11666	4

chrM	11266	12266	5



In [71]:
window_indx = np.array([window.count for window in binned_dnase])

In [49]:
first_window_indxs = np.where(window_indx == 1)[0]

In [51]:
last_window_indxs = first_window_indxs[1:] - 1

In [75]:
indxs = set(range(len(window_indx))) - set(last_window_indxs)

In [78]:
binned_dnase.at(list(indxs))

<BedTool(/tmp/pybedtools.tT3HYE.tmp)>

In [55]:
last_window_indxs

array([     1,      6,     19, ..., 377642, 377643, 377644])

In [53]:
last_windows_bed = binned_dnase.at(last_window_indxs)

In [54]:
last_windows_bed.head()

chrM	600	1097	2
 chrM	3608	4182	5
 chrM	16066	16537	13
 chrM	5259	5576	2
 chrM	6751	6767	2
 chr1	91852744	91853189	1
 chrM	8328	8796	1
 chr6	58778859	58779375	5
 chr21	9835596	9835739	18
 chrM	8044	8281	2
 

In [107]:
i=last_windows_bed[0]
i.stop = 1099
last_windows_bed[0] = i
last_windows_bed.head()

TypeError: 'BedTool' object does not support item assignment

In [109]:
type(last_windows_bed)

pybedtools.bedtool.BedTool

In [84]:
def pad_interval(interval, interval_size):
    length = interval.stop - interval.start
    if length < interval_size / 2.0:
        return False
    else:
        interval.stop += interval_size - length
        return interval

In [85]:
padded_last_window_bed = last_windows_bed.each(pad_interval, 1000)

In [86]:
padded_last_window_bed[0]

Interval(chrM:3608-4608)

In [62]:
slopped_binned_dnase = binned_dnase.slop(g='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes', b=50)

In [63]:
slopped_binned_dnase.head()

chrM	0	1050
 chrM	550	1147
 chrM	1158	2258
 chrM	1758	2858
 chrM	2358	3458
 chrM	2958	4058
 chrM	3558	4232
 chrM	8816	9916
 chrM	9416	10516
 chrM	10016	11116
 